In [ ]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats
from keras.models import Sequential, load_model, Model 
from keras.layers import Dense, Activation, Flatten, Dropout, Input, BatchNormalization, Activation, add, MaxPooling1D, Cropping1D 
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam
#from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
#import hyperopt.fmin as hypfmin
import keras
import tensorflow as tf
import random
import os
import pickle
import re
from math import ceil
from Bio import SeqIO
import warnings 
import datetime
import keras.backend as kb
from sklearn.metrics import average_precision_score, precision_recall_curve
import pybedtools as bt
import h5py
from keras.layers.merge import concatenate
from keras.layers.core import Lambda
import itertools

In [ ]:
def readM6A(File_list):
    tabs=pd.DataFrame()
    for f in File_list:
        tab=pd.read_csv(f,sep='\t',comment='#',header=None)
        tabs=tabs.append(tab,ignore_index=True)
    tabs.drop_duplicates(subset=[0,1,2],keep='first',inplace=True)
    return tabs 

In [ ]:
#need to add so as only take Pos samples that have RRACH 
def getInputMtxs_DNA(genomeFasta,sites,TrxtsGenomeFlank,chrs):
    fasta_sequences = SeqIO.parse(open(genomeFasta),'fasta')
    seqs={}
    for seq in fasta_sequences:
        if seq.name[:3]=='chr':
            seqs[seq.name]=seq  
    trxts=sites
    trxts=trxts.copy()
    chrs=['chr'+str(c) for c in chrs]
    trxts=trxts[trxts[0].isin(chrs)]
    trxts[1]=(trxts[2]-TrxtsGenomeFlank)
    trxts[2]=(trxts[2]+TrxtsGenomeFlank)
    trnMtxs=[]
    negBases=dict(zip(['A','C','G','T'],range(4)))
    posBases=dict(zip(['T','G','C','A'],range(4)))
    for r in range(len(trxts)):
        row=trxts.iloc[r,:]
        trxtW=row[2]-row[1]+1
        trxtSeq=seqs[row[0]].seq[row[1]-1:row[2]]
        trnMtx = np.matrix(np.zeros((4,trxtW)))
        if row[5] == '+':
            for bi in range(trxtW):
                try:
                    b = trxtSeq[bi]
                    trnMtx[posBases[b],bi] = 1
                except:
                    continue 
        else:
            for bi in range(trxtW):        
                try:
                    b = trxtSeq[bi]            
                    trnMtx[negBases[b],bi] = 1
                except:
                    continue 
            trnMtx = np.fliplr(trnMtx)
        trnMtxs.append(trnMtx)
    return trnMtxs

In [3]:
def verifyMotif(genomeFasta,sites,motif):
    fasta_sequences = SeqIO.parse(open(genomeFasta),'fasta')
    seqs={}
    for seq in fasta_sequences:
        if seq.name[:3]=='chr':
            seqs[seq.name]=seq 
    sites=sites.copy()
    TrueInds = [] 
    for r in range(len(sites)):
        row=sites.iloc[r,:]
        i=row[1]
        chrom = row[0]
        if chrom not in seqs.keys():
            continue 
        if row[5] == '+':
            if seqs[chrom].seq[i] == 'A':
                if seqs[chrom].seq[i+1] == 'C' :
                    if seqs[chrom].seq[i-1] in ['A','G'] :
                        if motif == 'RRACH':
                            if seqs[chrom].seq[i-2] in ['A','G'] :
                                if seqs[chrom].seq[i+2] in ['A','C','T'] :
                                    TrueInds.append(r)
                        elif motif == 'RAC' :
                            TrueInds.append(r)
        else:
            if seqs[chrom].seq[i] == 'T':
                if seqs[chrom].seq[i-1] == 'G' :
                    if seqs[chrom].seq[i+1] in ['T','C'] :
                        if motif == 'RRACH':
                            if seqs[chrom].seq[i+2] in ['T','C'] :
                                if seqs[chrom].seq[i-2] in ['T','G','A'] :
                                    TrueInds.append(r)
                        elif motif == 'RAC' :
                            TrueInds.append(r)
    return sites.iloc[TrueInds,:]

In [ ]:

#WHICH TRANSCRIPTS WE USE COULD AFFECT WHICH SITES Neg TRAINING DATA IS ON and EFFECT OUTCOME --- may need to adjust this
def processGff(AgFn) :
    GCDE = pd.read_csv(AgFn,sep='\t',comment='#',header=None)
    trxts = GCDE[GCDE[2]=="transcript"]
    pcgBool=[bool(re.search('gene_type \"(?:protein_coding|.*l.*ncRNA)\";',s)) for s in trxts[8]]
    trxts=trxts[pcgBool]
    trxts['trx_name']=trxts.iloc[:,8].str.extract('transcript_name \"(.+?)\";')
    trxts['gene']=trxts.iloc[:,8].str.extract('gene_name \"(.+?)\";')
    trxts['trxSz']=trxts[4]-trxts[3]
    splitGps = trxts.groupby('gene')
    dups=pd.DataFrame()
    trxts=pd.DataFrame()
    for sg in splitGps:
        sg=sg[1]
        if len(sg) > 1:
            sg=sg[sg['trxSz'] == sg['trxSz'].max()]
            if len(sg) > 1:
                sg=sg.iloc[0:1,:]
                if len(sg) > 1:
                    dups=dups.append(sg)
                    continue 
        trxts=trxts.append(sg)
    if len(dups) > 0:
        print("{} Removed (Still has duplicates!!!)".format(dups['trx_name']))
        print("{} gene(s) with duplicates removed!!!".format(len(set(dups['gene']))))
    cols=list(range(0,9))+['trx_name','gene']
    trxts=trxts.loc[:,cols]
    return trxts

In [ ]:
#add ratio for neg sites per transcript? 
#add greater flank on sides of transcript for neg samples ?
def negsOutsidePeaks(genomeFasta,m6A_peaks,m6A_sites,trxts,motif) :
    fasta_sequences = SeqIO.parse(open(genomeFasta),'fasta')
    seqs={}
    for seq in fasta_sequences:
        if seq.name[:3]=='chr':
            seqs[seq.name]=seq 
    m6A_peaks = bt.BedTool.from_dataframe(m6A_peaks)
    m6A_sites = bt.BedTool.from_dataframe(m6A_sites)
    trxts = bt.BedTool.from_dataframe(trxts.loc[:,list(range(0,8))+['trx_name']])
    #make sure only use transcripts that overlap with an m6aSite
    trxts = trxts.intersect(m6A_sites,wa=True,s=True)
    overlaps = trxts.intersect(m6A_peaks,wb=True,wa=True,s=True)
    overlaps = overlaps.to_dataframe(sep='\t',comment='#',header=None).drop_duplicates().groupby(8)
    chroms, Ns, trx_names, strands= [],[],[],[]
    for sg in overlaps:
        sg=sg[1]
        trxtRng=pd.Series(range(sg.iloc[0,3]-1+2,sg.iloc[0,4]-2))
        pkRngs=list(itertools.chain.from_iterable(
            [list(range(sg.iloc[r,10]-1,sg.iloc[r,11])) for r in range(len(sg))]))
        trxtRng=list(trxtRng[~trxtRng.isin(pkRngs)])
        negSites=[]
        chrom = sg.iloc[0,0]
        if sg.iloc[0,6] == '+':
            for i in trxtRng:
                if seqs[chrom].seq[i] == 'A':
                    if seqs[chrom].seq[i+1] == 'C' :
                        if seqs[chrom].seq[i-1] in ['A','G'] :
                            if motif == 'RRACH':
                                if seqs[chrom].seq[i-2] in ['A','G'] :
                                    if seqs[chrom].seq[i+2] in ['A','C','T'] :
                                        negSites.append(i)
                            elif motif == 'RAC' :
                                negSites.append(i)
        else:
            for i in trxtRng:
                if seqs[chrom].seq[i] == 'T':
                    if seqs[chrom].seq[i-1] == 'G' :
                        if seqs[chrom].seq[i+1] in ['T','C'] :
                            if motif == 'RRACH':
                                if seqs[chrom].seq[i+2] in ['T','C'] :
                                    if seqs[chrom].seq[i-2] in ['T','G','A'] :
                                        negSites.append(i)
                            elif motif == 'RAC' :
                                negSites.append(i)
        lns = len(negSites)
        chroms = chroms + [chrom]*lns
        Ns = Ns + negSites
        trx_names = trx_names + [sg.iloc[0,8]]*lns
        strands = strands + [sg.iloc[0,6]]*lns
    allNegs = pd.DataFrame([chroms, Ns, pd.Series(Ns)+1, trx_names, [1]*len(Ns), strands]).T
    return allNegs


In [ ]:
def Mtxs_toH5Blocks(InpMtxs,TgtMtxs,CHUNK_SIZE,train_or_test):
    num_CHUNKS=ceil(len(InpMtxs)/CHUNK_SIZE)
    h5f_Inp=h5py.File('Inp_'+train_or_test+'.h5', 'w')
    h5f_Tgt=h5py.File('Tgt_'+train_or_test+'.h5', 'w')
    for i in range(num_CHUNKS):
        Ib=InpMtxs[CHUNK_SIZE*i:CHUNK_SIZE*(i+1)]
        Tb=TgtMtxs[CHUNK_SIZE*i:CHUNK_SIZE*(i+1)]
        h5f_Inp[str(i)]=np.swapaxes(np.stack(np.array(Ib),axis=1).T,0,1).astype('int8')
        h5f_Tgt[str(i)]=np.expand_dims(np.expand_dims(np.array(Tb),1),2).astype('int8')
    print ('Inp_'+train_or_test+'.h5'+'and'+'Tgt_'+train_or_test+'.h5'+'created')

In [ ]:
def randSubBlocks(InpBlocks,TgtBlocks):
    InpBlocks, TgtBlocks = pd.Series(InpBlocks), pd.Series(TgtBlocks)
    rand = np.random.permutation(len(InpBlocks))
    InpBlocks = list(InpBlocks.iloc[rand])
    TgtBlocks = list(TgtBlocks.iloc[rand])
    return InpBlocks,TgtBlocks

In [ ]:
#create multiple resnet layers in model 
def resnet(x,fltrNumb,fltrW,dilr,numbRns):
    def oneLoop(x):
        z = BatchNormalization()(x)
        z = Activation('relu')(z)
        z = Conv1D(fltrNumb,kernel_size=(fltrW,),dilation_rate=dilr,padding='same')(z)
        return z
    full=x
    for n in range(numbRns):
        z=oneLoop(full)
        z=oneLoop(z)
        full = add([full,z]) 
    return full   

In [ ]:
def make_parallel(model, gpu_count):

    def get_slice(data, idx, parts):

        shape = tf.shape(data)
        stride = tf.concat([shape[:1]//parts, shape[1:]*0], 0)
        start = stride * idx

        size = tf.concat([shape[:1]//parts, shape[1:]], 0) 
        # Split the batch into equal parts 

        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    # Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                # Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape,
                                  arguments={'idx': i, 'parts': gpu_count})(x)
                    inputs.append(slice_n)

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                # Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # Merge outputs on CPU
    with tf.device('/cpu:0'):
        
        merged = []
        for outputs in outputs_all:
            merged.append(concatenate(outputs, axis=0))
            
        return Model(inputs=model.inputs, outputs=merged)


In [ ]:
def topK_auPRC(model,Inp,Tgt,numOut_Classes):
    p=model.predict(Inp)
    Tks=[]
    PRs=[]
    if numOut_Classes > 1:
        cRange=range(1,numOut_Classes)
    else:
        cRange=[0]
    for c in cRange:
        Fullc=Tgt[:,:,c].flatten()
        S=[i for i in range(len(Fullc)) if Fullc[i] == 1]
        pS=np.argsort(p[:,:,c].flatten())[-len(S):]
        pS=pd.Series(pS)
        nCrct=pS[pS.isin(S)].size
        Tk=nCrct/float(len(S))
        Tks.append(Tk)
        PR=average_precision_score(Fullc,p[:,:,c].flatten())
        PRs.append(PR)
    if numOut_Classes < 3:
        Tks.append(0.0)
        PRs.append(0.0)
    return(Tks[0],Tks[1],PRs[0],PRs[1])